In [11]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import warnings

# warnings.filterwarnings("ignore")

%matplotlib inline

In [25]:
df = pd.read_csv('data/google_cleaned.csv')
df.head(2)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Current Ver,Android Ver,last_updated_day,last_updated_month,last_updated_year
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000.0,10000,Free,0.0,Everyone,Art & Design,1.0.0,4.0.3 and up,7,1,2018
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,2.0.0,4.0.3 and up,15,1,2018


In [26]:
# Found the same application has duplicates
print('Has duplicates' if len(df.duplicated('App').unique()) > 1 else 'No Duplicates')

# Observed 1181 cases of duplicated rows
print("number of duplicates:", df[df.duplicated('App')].shape[0])

# Drop duplicates
df = df.drop_duplicates(subset=['App'], keep='first')

print('Has duplicates' if len(df.duplicated('App').unique()) > 1 else 'No Duplicates')



Has duplicates
number of duplicates: 1181
No Duplicates
